# Итоговый проект по курсу "ML для финансового анализа"

*Автор - Карданов М. Т.*

## Задача проекта:

Разработать торгового робота, представляющего из себя ансамбль моделей технического анализа и машинного обучения, анализирующего данные о стоимости акций компании Google за последний год и формулирующего торговые действия.

## План проекта:

1. Загрузка данных;
1. Feature engineering;
1. Построение модели технического анализа;
1. Построение модели машинного обучения;
1. Обучение ансамбля;
1. Проверка качества ансамбля на тестовой выборке;
1. Составление презентации, защита.

## 0: Импорты

In [862]:
import pandas as pd
import sklearn
import numpy as np
import yfinance
import os
import warnings
from backtesting import Strategy, Backtest
from sklearn.ensemble import GradientBoostingClassifier

In [863]:
RANDOM_STATE = 12345

## 1: Загрузка данных

In [864]:
def download_data() -> pd.DataFrame:

    data = yfinance.download(tickers="GOOG", 
                             period="5y", 
                             interval="1d", 
                             multi_level_index=False)
    data.index = pd.to_datetime(data.index)

    return data

## 2: Feature-engineering

In [865]:
def add_date_features(data: pd.DataFrame) -> pd.DataFrame:

    data["Date_time"] = data.index

    data['Year']=data['Date_time'].dt.strftime('%Y')
    data['Month']=data['Date_time'].dt.strftime('%m')
    data['Day']=data['Date_time'].dt.strftime('%d')

    return data

В качестве признаков технического анализа я решил выбрать 5-ти, 10-ти и 20-барные MA и EMA, в основном из-за того, что они просты в реализации и, одновременно с этим, несут в себе много информации о более долгосрочном поведении цены.

In [866]:
def add_price_features(data: pd.DataFrame) -> pd.DataFrame:

    data["5_day_MA"] = data["Close"].rolling(window=5, min_periods=1).mean()
    data["10_day_MA"] = data["Close"].rolling(window=10, min_periods=1).mean()
    data["20_day_MA"] = data["Close"].rolling(window=20, min_periods=1).mean()

    data["5_day_EMA"] = data["Close"].ewm(span=5, min_periods=1).mean()
    data["10_day_EMA"] = data["Close"].ewm(span=10, min_periods=1).mean()
    data["20_day_EMA"] = data["Close"].ewm(span=20, min_periods=1).mean()

    return data

In [867]:
def add_target(data: pd.DataFrame) -> pd.DataFrame:
    
    data["Target"] = pd.Series(0)
    data["Close_in_3_days"] = data["Close"].shift(-3)

    data.loc[(data["Close_in_3_days"] - data["Close"]) >= 3, "Target"] = 3
    data.loc[(data["Close_in_3_days"] - data["Close"]) <= -3, "Target"] = 1
    data.loc[(-3 < (data["Close_in_3_days"] - data["Close"])) & ((data["Close_in_3_days"] - data["Close"]) < 3), "Target"] = 2

    data = data.dropna(subset=["Target"])

    data = data.drop("Close_in_3_days", axis=1)

    return data

In [868]:
def save_data(data: pd.DataFrame, filename: str = "data/processed_data.csv") -> None:

    if os.path.exists(filename):
        data.to_csv(filename, mode='a', header=False, index=False)
    else:
        data.to_csv(filename, mode='w', header=True, index=False)

In [869]:
def load_data():
    data = download_data()
    data = add_date_features(data)
    data = add_price_features(data)
    data = add_target(data)
    save_data(data)

## 3: Модель технического анализа (**ДОДЕЛАТЬ**)

В качестве модели технического анализа я выбрал стохастический осцилятор, поскольку, **ДОБАВИТЬ**

In [870]:
# class StochOscilatorStrategy(Strategy):

#     def init(self):
#         self.signal = self.I(lambda: self.data.Signal)
#         self.previous_signal = 0
#         self.size = 0.1

#     def next(self):
#         current_signal = self.signal[-1]

#         if current_signal != self.previous_signal:
#             if current_signal == 1:
#                 if self.position.is_short:
#                     self.position.close()
                    
#                 if not self.position.is_long:
#                     self.buy(size=self.size)
                    
#             elif current_signal == -1:
#                 if self.position.is_long:
#                     self.position.close()
                   
#                 if not self.position.is_short:
#                     self.sell(size=self.size)
                    
#             elif current_signal == 0:
#                 if self.position:
#                     self.position.close()

#         self.previous_signal = current_signal  

## 4: Модель машинного обучения

Мне кажется, что мне, как начинающему кванту, можно для начала построить систему с не-нейросетевой моделью, чтобы на ограниченных ресурсах, имеющихся в моём распоряжении, добиться, насколько это возможно, быстрой выдачи качественных предсказаний. Поэтому я остановился на модели градиентного бустинга из библиотеки sklearn.

In [871]:
def prepare_data_for_training(val_days: int=500) -> list:

    data = pd.read_csv("data/processed_data.csv")
    data["Date_time"] = pd.to_datetime(data["Date_time"])

    test_start_date = data["Date_time"].max() - pd.DateOffset(days=val_days)

    train_data = data[data["Date_time"] < test_start_date]
    test_data = data[data["Date_time"] >= test_start_date]

    train_data = train_data.drop("Date_time", axis=1)
    test_data = test_data.drop("Date_time", axis=1)

    y_train = train_data["Target"]
    y_test = test_data["Target"]
    X_train = train_data.drop("Target", axis=1)
    X_test = test_data.drop("Target", axis=1)

    return [X_train, y_train, X_test, y_test]

In [872]:
def train_model(X_train: pd.DataFrame, y_train: pd.Series) -> GradientBoostingClassifier:

    model = GradientBoostingClassifier(learning_rate=0.1, random_state=RANDOM_STATE, n_estimators=50)

    model.fit(X_train, y_train)

    return model

In [873]:
def test_model(X_test: pd.DataFrame, y_test: pd.Series, model: GradientBoostingClassifier) -> GradientBoostingClassifier:

    preds = model.predict(X_test)

    test_acc = sklearn.metrics.accuracy_score(y_test, preds)
    test_prec = sklearn.metrics.precision_score(y_test, preds, average="weighted")
    test_rec = sklearn.metrics.recall_score(y_test, preds, average="weighted")

    print(f"Точность модели на тестовой выборке составила {test_acc}")
    print(f"Precision модели на тестовой выборке составила {test_prec}")
    print(f"Recall модели на тестовой выборке составил {test_rec}")

    return model

In [874]:
def train_evaluate() -> GradientBoostingClassifier:
    data_list = prepare_data_for_training()
    trained_model = train_model(X_train=data_list[0], y_train=data_list[1])
    tested_model = test_model(X_test=data_list[2], y_test=data_list[3], model=trained_model)
    return tested_model

In [875]:
def pipeline():
    load_data()
    model = train_evaluate()
    return model

In [876]:
pipeline()

C:\Users\mtkar\AppData\Local\Temp\ipykernel_20528\1684380392.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yfinance.download(tickers="GOOG",
[*********************100%***********************]  1 of 1 completed


Точность модели на тестовой выборке составила 0.28362573099415206
Precision модели на тестовой выборке составила 0.4105427007473791
Recall модели на тестовой выборке составил 0.28362573099415206


,loss,'log_loss'
,learning_rate,0.1
,n_estimators,50
,subsample,1.0
,criterion,'friedman_mse'
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_depth,3
,min_impurity_decrease,0.0
,init,None
